# Machine Learning - UFRJ 2022.2
## Limpeza, Normalização e Padronização

#### Feito por Rafael da Silva Fernandes, DRE: 117196229

## Bibliotecas

In [1]:
import pandas as pd
import numpy as np

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, Normalizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install scikit-posthocs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import scikit_posthocs as sp

## Dataset

In [4]:
df = pd.read_csv("data-t1.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


## 1 - Limpar dataset

### Atributos

In [5]:
df["diagnosis"] = np.where(df["diagnosis"] == "M", 1, 0)
df.drop(columns = ["Unnamed: 32"], inplace = True)

In [6]:
df_var = df.var().to_frame()
df_var.drop(index = ['id'], inplace = True)
df_var.columns = ['Var']
attributes = df_var.loc[df_var['Var'] > 1].index.tolist()
attributes

['radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'perimeter_se',
 'area_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst']

In [7]:
df_0 = df[attributes]
df_0['id'] = df['id']
df_0['diagnosis'] = df['diagnosis']
df_0.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,perimeter_se,area_se,radius_worst,texture_worst,perimeter_worst,area_worst,id,diagnosis
0,17.99,10.38,122.80,1001.0,8.589,153.40,25.38,17.33,184.60,2019.0,842302,1
1,20.57,17.77,132.90,1326.0,3.398,74.08,24.99,23.41,158.80,1956.0,842517,1
2,19.69,21.25,130.00,1203.0,4.585,94.03,23.57,25.53,152.50,1709.0,84300903,1
3,11.42,20.38,77.58,386.1,3.445,27.23,14.91,26.50,98.87,567.7,84348301,1
4,20.29,14.34,135.10,1297.0,5.438,94.44,22.54,16.67,152.20,1575.0,84358402,1


### Valores nulos

In [8]:
print(f"Tamanho do dataset antes de eliminar duplicatas e valores nulos = {df_0.shape}")

df_0.drop_duplicates(inplace = True)
df_0[~df_0.isin([np.nan, np.inf, -np.inf]).any(1)]
df_0.dropna(inplace = True)

print(f"Tamanho do dataset após eliminar duplicatas e valores nulos = {df_0.shape}")

Tamanho do dataset antes de eliminar duplicatas e valores nulos = (569, 12)
Tamanho do dataset após eliminar duplicatas e valores nulos = (569, 12)


## 2 - Normalização e Padronização

In [9]:
results = {}

### a) Detecção e Remoção de Outliers → Padronização → Normalização

Remoção de outlier

In [10]:
df_1 = df_0[(np.abs(stats.zscore(df_0)) < 3).all(axis = 1)]

Divisão em treino e teste

In [11]:
x_1 = df_1.iloc[:, :-2]
y_1 = df_1["diagnosis"]

x_1_train, x_1_test, y_1_train, y_1_test = train_test_split(
    x_1, y_1, test_size = 1/3, random_state = 4510471
)

#### Regressão Linear

In [12]:
pipe1_reglin = make_pipeline(
    StandardScaler(),
    Normalizer(),
    LinearRegression()
)

pipe1_reglin.fit(x_1_train, y_1_train)

results['Reg Lin OUT->P->N'] = pipe1_reglin.score(x_1_test, y_1_test)

#### Regressão Logística

In [13]:
pipe1_reglog = make_pipeline(
    StandardScaler(),
    Normalizer(),
    LogisticRegression()
)

pipe1_reglog.fit(x_1_train, y_1_train)

results['Reg Log OUT->P->N'] = pipe1_reglog.score(x_1_test, y_1_test)

### b) Padronização → Normalização

Divisão em treino e teste

In [14]:
x_2 = df_0.iloc[:, :-2]
y_2 = df_0["diagnosis"]

x_2_train, x_2_test, y_2_train, y_2_test = train_test_split(
    x_2, y_2, test_size = 1/3, random_state = 4510471
)

#### Regressão Linear

In [15]:
pipe2_reglin = make_pipeline(
    StandardScaler(),
    Normalizer(),
    LinearRegression()
)

pipe2_reglin.fit(x_2_train, y_2_train)

results['Reg Lin P->N'] = pipe2_reglin.score(x_2_test, y_2_test)

#### Regressão Logística

In [16]:
pipe2_reglog = make_pipeline(
    StandardScaler(),
    Normalizer(),
    LogisticRegression()
)

pipe2_reglog.fit(x_2_train, y_2_train)
results['Reg Log P->N'] = pipe2_reglog.score(x_2_test, y_2_test)

### c) Normalização → Padronização

#### Regressão Linear

In [17]:
pipe3_reglin = make_pipeline(
    Normalizer(),
    StandardScaler(),
    LinearRegression()
)

pipe3_reglin.fit(x_2_train, y_2_train)

results['Reg Lin N->P'] = pipe3_reglin.score(x_2_test, y_2_test)

#### Regressão Logística

In [18]:
pipe3_reglog = make_pipeline(
    Normalizer(),
    StandardScaler(),
    LogisticRegression()
)

pipe3_reglog.fit(x_2_train, y_2_train)

results['Reg Log N->P'] = pipe3_reglog.score(x_2_test, y_2_test)

## 3 - Avaliação dos resultados com parâmetros default

In [19]:
df_results = pd.DataFrame.from_dict(results, orient = 'index').T
df_results = df_results[
    [
        'Reg Lin OUT->P->N', 'Reg Lin P->N', 'Reg Lin N->P',
        'Reg Log OUT->P->N', 'Reg Log P->N', 'Reg Log N->P'
    ]
]
df_results

,Reg Lin OUT->P->N,Reg Lin P->N,Reg Lin N->P,Reg Log OUT->P->N,Reg Log P->N,Reg Log N->P
0,0.795653,0.825002,0.672521,0.955556,0.952632,0.952632
